# Exercícios - Fontes de dados 💾

## 🔎 Você é uma cientista de dados e cidadã consciente do seu dever de monitorar os poderes legislativo e executivo quanto à utilização dos recursos públicos. 

### Uma forma bastante interessante de se obter dados para realizar o seu trabalho é utilizando as APIs dos portais de dados abertos disponibilizados pelos órgãos públicos. 

### 👀 De olho nas despesas dos deputados federais!
A Câmara dos Deputados, como vimos na aula de hoje, disponibiliza o portal **Dados Abertos**, que provê diferentes conjuntos de dados, incluindo detalhes sobre atividades legislativas, deputados e suas despesas, junto com uma API que permite ao usuário recuperar dados de forma programática. Você pode navegar pelo portal através da seguinte URL: 
```
https://dadosabertos.camara.leg.br/
```
Nele, você poderá obter informações sobre como utilizá-lo, conhecer os dados disponíveis, acessar a documentação da API e entender como ela funciona. Para acessar diretamente a documentação da API, utilize a URL abaixo:
```
https://dadosabertos.camara.leg.br/swagger/api.html
```
Antes de iniciar os exercícios abaixo, navegue pela documentação da API; vá até a seção que lista os serviços que permitem acessar os dados dos **deputados**, prestando atenção na URL de acesso e nos parâmetros obrigatórios de cada um deles (se existirem).

<img src='./img/loading-thinking.gif' width='20%'>

Ok, agora vamos aos exercícios! 💪🏻 Queremos analisar as despesas realizadas pelos deputados da legislatura atual eleitos por um determinado  Estado. Vimos como obter a lista completa dos 513 deputados. Crie uma função denominada **listar_deputados_por_estado**, que receberá como parâmetro a sigla de um Estado brasileiro e retornará uma lista contendo todos os deputados federais eleitos por esse Estado.

In [1]:
import requests

In [2]:
def read_api(url, params=None):
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        content = response.json()
        return content

In [3]:
def listar_deputados_por_estado(sigla_uf):
    url_api_dep = 'https://dadosabertos.camara.leg.br/api/v2/deputados'
    params={'siglaUf':f'{sigla_uf}'}
    data = read_api(url_api_dep, params)
    if data:
        return data['dados']    

Quantos deputados representam o estado de Minas Gerais?

In [4]:
dep_mg = listar_deputados_por_estado('MG')

In [5]:
print(len(dep_mg))

53


Agora, crie uma função denominada **despesas_por_deputado**, que receberá como parâmetro o identificador (id) de um parlamentar e retornará a lista de despesas desse parlamentar **nos últimos seis meses**.

<p>Obs.: O intervalo de seis meses anteriores à requisição é a resposta padrão da API quando o período não é informado como parâmetro, conforme informado pela documentação, portanto, para este exercício, não há necessidade de manipular datas.

In [28]:
#seleciona o id e o nome do primeiro deputado da lista para teste
dep_id = dep_mg[0]['id']
dep_nome = dep_mg[0]['nome']
print(dep_id)
print(dep_nome)

74646
Aécio Neves


In [41]:
def despesas_por_deputado(idDep):
    url_api_dep = f'https://dadosabertos.camara.leg.br/api/v2/deputados/{idDep}/despesas'
    
    data = read_api(url_api_dep)
    if data:
        return data['dados']

🌶️ Desafio! Faça a consulta de despesas para um deputado qualquer da lista de deputados que representam o estado de Minas Gerais. Qual o valor total das despesas do deputado no período consultado (seis meses anteriores à data da consulta)? Qual o maior valor pago no período e para qual empresa fornecedora?

In [44]:
desp_dep = despesas_por_deputado(dep_id)
print(desp_dep)

[{'ano': 2023, 'mes': 3, 'tipoDespesa': 'MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE PARLAMENTAR', 'codDocumento': 7520058, 'tipoDocumento': 'Recibos/Outros', 'codTipoDocumento': 1, 'dataDocumento': '2023-03-31', 'numDocumento': '71897', 'valorDocumento': 6515.4, 'urlDocumento': 'https://www.camara.leg.br/cota-parlamentar/documentos/publ/233/2023/7520058.pdf', 'nomeFornecedor': 'ANUAR DONATO CONSULT. IMOBILIARIA', 'cnpjCpfFornecedor': '04292201000160', 'valorLiquido': 6515.4, 'valorGlosa': 0.0, 'numRessarcimento': '', 'codLote': 1920596, 'parcela': 0}, {'ano': 2023, 'mes': 5, 'tipoDespesa': 'MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE PARLAMENTAR', 'codDocumento': 7553631, 'tipoDocumento': 'Recibos/Outros', 'codTipoDocumento': 1, 'dataDocumento': '2023-05-25', 'numDocumento': '75319', 'valorDocumento': 6515.4, 'urlDocumento': 'https://www.camara.leg.br/cota-parlamentar/documentos/publ/233/2023/7553631.pdf', 'nomeFornecedor': 'ANUAR DONATO CONSULT. IMOBILIARIA', 'cnpjCpfFornecedor':

In [95]:
def calcula_valor_total_despesas(lista_desp):
    valor_total = sum([desp['valorLiquido'] for desp in lista_desp])
    return valor_total

In [96]:
valor_total = calcula_valor_total_despesas(desp_dep)
print(f'O deputado {dep_nome} gastou R$ {valor_total:.2f} de cota parlamentar nos ultimos 6 meses')

O deputado Aécio Neves gastou R$ 43505.99 de cota parlamentar nos ultimos 6 meses


In [97]:
from operator import itemgetter

def retorna_maior_valor_gasto(lista_desp):
    lista_desp_ord = sorted(lista_desp, key=itemgetter('valorLiquido'), reverse=True)
    val_maior_desp = lista_desp_ord[0]['valorDocumento']
    empresa = lista_desp_ord[0]['nomeFornecedor']
    return val_maior_desp,empresa


In [98]:
val_maior_desp, empresa = retorna_maior_valor_gasto(desp_dep)
print(f'O maior valor gasto pelo deputado {dep_nome} foi R$ {val_maior_desp:.2f}, pago à empresa {empresa}.')

O maior valor gasto pelo deputado Aécio Neves foi R$ 7566.27, pago à empresa ANUAR DONATO CONSULT. IMOBILIARIA.


🌶️🌶️ Duplo desafio! Faça uma função que receberá como parâmetro a sigla de um Estado e retornará uma lista de dicionários contendo o nome do deputado e sua respectiva lista de despesas.<p>
*Dica*: Ao acessar serviços de uma API, é essencial que implementemos mecanismos para garantir que não o estejamos sobrecarregando ou violando os seus limites de taxas de requisição. Muitos servidores limitam o número de requisições feitas por um mesmo endereço IP em um espaço específico de tempo, por exemplo, 15 requisições por min., e se um programa de computador exceder esses limites, ele pode ser temporariamente bloqueado ou permanentemente suspenso de acessar a API. A documentação da API que estamos utilizando não nos informa a sua taxa limite de requisições, mas podemos implementar preventivamente um mecanismo para evitar o bloqueio do seu programa. Uma das formas de se fazer isso é inserir um timer com um determinado atraso (normalmente de 60 segundos, mas depende do serviço) quando o número de requisições que você estiver fazendo chegar a um limite. Faça uma pesquisa sobre como inserir um timer em sua função. Determine um tempo de atraso de 60 seg. e um número máximo de disparo de requisições de 15.

In [93]:
import time

DELAY_IN_SECONDS = 60

def lista_despesas_deputados_por_estado(siglaUf):
    dep_por_estado = listar_deputados_por_estado(siglaUf)
    cont = 0
    lista_dep_desp = []
    for dep in dep_por_estado:
        #print(dep['id']
        cont += 1
        despesas = despesas_por_deputado(dep['id'])
        if despesas:
            novo_dep = {'deputado':dep['nome'], 'despesas':despesas}
            lista_dep_desp.append(novo_dep)
        if cont == 15:
            time.sleep(DELAY_IN_SECONDS)
            count = 0
    return lista_dep_desp

In [90]:
lista_desp_dep = lista_despesas_deputados_por_estado('MG')

In [92]:
len(lista_desp_dep)

53

Se conseguiu implementar a função, busque as despesas dos deputados eleitos pelo estado de Minas Gerais e tente responder às seguintes perguntas: que deputado gastou mais (em valores totais) durante o período pesquisado? Qual foi o maior valor de despesa única entre todos os deputados?

In [108]:
dep = lista_desp_dep[0]
nome_dep = dep['deputado']
maior_valor_total = calcula_valor_total_despesas(dep['despesas'])
for i in range(1,len(lista_desp_dep)):
    dep = lista_desp_dep[i]
    valor_atual = calcula_valor_total_despesas(dep['despesas'])
    if maior_valor_total < valor_atual:
        nome_dep = dep['deputado']
        maior_valor_total = valor_atual

print(f'O deputado {nome_dep} foi o que mais gastou durante o periodo pesquisado, um total de R$ {maior_valor_total:.2f}.')   

O deputado Mário Heringer foi o que mais gastou durante o periodo pesquisado, um total de R$ 104758.10.


In [111]:
lista_maiores_valores_gastos_por_deputado = []
for dep in lista_desp_dep:
    valor, empresa = retorna_maior_valor_gasto(dep['despesas'])
    lista_maiores_valores_gastos_por_deputado.append({'deputado':dep['deputado'],'valor':valor,'empresa':empresa})
lista_ordenada = sorted(lista_maiores_valores_gastos_por_deputado, key=itemgetter('valor'), reverse=True)
nome_dep = lista_ordenada[0]['deputado']
valor = lista_ordenada[0]['valor']
empresa = lista_ordenada[0]['empresa']
print(f'O deputado {nome_dep} foi o que gastou o maior valor em uma única despesa durante o periodo pesquisado, um total de R$ {valor:.2f} para a empresa {empresa}.') 

O deputado Hercílio Coelho Diniz foi o que gastou o maior valor em uma única despesa durante o periodo pesquisado, um total de R$ 30000.00 para a empresa FRAMIN - AGENCIA DE COMUNICACAO LTDA.
